- Name: Isaac Ndirangu Muturi
- Email: ndirangumuturi749@gmail.com

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

# 1. Use a generator

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


- **Question 1: What is the sum of the outputs of the generator for limit = 5?**

In [2]:
limit = 5
generator = square_root_generator(limit)
sum = 0
for value in generator:
    sum += value
print(sum)

8.382332347441762


- **Question 2: What is the 13th number yielded**

In [3]:
limit = 13
generator = square_root_generator(limit)
for i, value in enumerate(generator):
    if i == 12:
        print(value)
        break

3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [16]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [10]:
# #Install the dependencies
# %%capture
# !pip install dlt[duckdb]

In [17]:
import dlt

# Define the pipeline
pipeline = dlt.pipeline(pipeline_name='people_pipeline',destination='duckdb', dataset_name='people_data')
# Load data from the first generator into DuckDB
info1 = pipeline.run(data=people_1(), table_name='people', write_disposition='replace')
print(info1)

# Append data from the second generator to the same table in DuckDB
info2 = pipeline.run(data=people_2(), table_name='people', write_disposition='append')
print(info2)


Pipeline people_pipeline load step completed in 1.77 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:////home/ndirangu749/ZOOMCAMP/Data_Engineering_Zoomcamp_2024/workshop_1_data_ingestion/Homework_workshop_1/people_pipeline.duckdb location to store data
Load package 1707923415.4487298 is LOADED and contains no failed jobs
Pipeline people_pipeline load step completed in 1.26 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:////home/ndirangu749/ZOOMCAMP/Data_Engineering_Zoomcamp_2024/workshop_1_data_ingestion/Homework_workshop_1/people_pipeline.duckdb location to store data
Load package 1707923417.667702 is LOADED and contains no failed jobs


In [31]:
import duckdb

# Connect to the database
conn = duckdb.connect("people_pipeline.duckdb")

conn.sql(f"SET search_path = 'people_data'")

# Display the available tables
tables_info = conn.execute("SHOW TABLES").fetch_df()
display(tables_info)

# Query the people table
query = "SELECT * FROM people_data.people"
people_info = conn.execute(query).fetch_df()
display(people_info)


quiz = "SELECT SUM(age) FROM people"
sum_ages = conn.sql(quiz).df()
display(sum_ages)

,name
0,_dlt_loads
1,_dlt_pipeline_state
2,_dlt_version
3,people


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707923415.4487298,1qaI4a9aWnyMmw,None
1,2,Person_2,27,City_A,1707923415.4487298,UPxCQzmH9ZVVfQ,None
2,3,Person_3,28,City_A,1707923415.4487298,KOg9EOmrIpBRxg,None
3,4,Person_4,29,City_A,1707923415.4487298,YxbwwkH40T/IVA,None
4,5,Person_5,30,City_A,1707923415.4487298,etJIP4QL0tC9+A,None
5,3,Person_3,33,City_B,1707923417.667702,wNOCzNDiuIU9lA,Job_3
6,4,Person_4,34,City_B,1707923417.667702,B4Ba01gjTeLP8A,Job_4
7,5,Person_5,35,City_B,1707923417.667702,ZgoKuHMvkLG65Q,Job_5
8,6,Person_6,36,City_B,1707923417.667702,SP4/04fPR94uvw,Job_6
9,7,Person_7,37,City_B,1707923417.667702,KNM5xbu5256A2Q,Job_7


,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [33]:
# Load data from the first generator into DuckDB
info1 = pipeline.run(data=people_1(), table_name="people_id_merge", write_disposition="replace", primary_key='ID')
print(info1)
# Load the second generator to the first table with merge
info2 = pipeline.run(data=people_2(), table_name="people_id_merge", write_disposition="merge", primary_key='ID')
print(info2)

Pipeline people_pipeline load step completed in 0.44 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:////home/ndirangu749/ZOOMCAMP/Data_Engineering_Zoomcamp_2024/workshop_1_data_ingestion/Homework_workshop_1/people_pipeline.duckdb location to store data
Load package 1707925042.4385152 is LOADED and contains no failed jobs
Pipeline people_pipeline load step completed in 1.37 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:////home/ndirangu749/ZOOMCAMP/Data_Engineering_Zoomcamp_2024/workshop_1_data_ingestion/Homework_workshop_1/people_pipeline.duckdb location to store data
Load package 1707925043.2646096 is LOADED and contains no failed jobs


In [34]:
import duckdb

# Connect to the database
conn = duckdb.connect("people_pipeline.duckdb")

conn.sql(f"SET search_path = 'people_data'")

# Display the available tables
tables_info = conn.execute("SHOW TABLES").fetch_df()
display(tables_info)

# Query the people table
query = "SELECT * FROM people_id_merge"
people_info = conn.execute(query).fetch_df()
display(people_info)


quiz = "SELECT SUM(age) FROM people_id_merge"
sum_ages = conn.sql(quiz).df()
display(sum_ages)

,name
0,_dlt_loads
1,_dlt_pipeline_state
2,_dlt_version
3,people
4,people_id
5,people_id_merge


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707925042.4385152,rmZIiTKUXGOkrw,None
1,2,Person_2,27,City_A,1707925042.4385152,msurKbqmRWnYYg,None
2,8,Person_8,38,City_B,1707925043.2646096,nK1LBsI3Jm43pg,Job_8
3,5,Person_5,35,City_B,1707925043.2646096,UGvQ/0tkh7aRPg,Job_5
4,4,Person_4,34,City_B,1707925043.2646096,4Ax3XhlxYHCx4w,Job_4
5,7,Person_7,37,City_B,1707925043.2646096,OHekt9437Sr/ow,Job_7
6,3,Person_3,33,City_B,1707925043.2646096,SqxdizqF6CB/og,Job_3
7,6,Person_6,36,City_B,1707925043.2646096,ceg9zPoXv87cMA,Job_6


,sum(age)
0,266.0
